# Subjektive Leistungseinschätzung
- Bubble chart
- Stacked Barchart

In [11]:
import pandas as pd

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Dictionary zur Speicherung der berechneten Werte
leistung_df = {}

# Durch alle Jahre iterieren
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen

    # Studiengänge automatisch erkennen
    studiengaenge = [col for col in df.columns if "Bachelor" in col or "Master" in col]

    # Extrahiere die Zeilen für die subjektive Leistungseinschätzung
    positiv_row = df[(df["Variable"] == "Subjektive Leistungseinschätzung") & (df["Category"] == "Positiv Einschätzung (%)")]
    indifferent_row = df[(df["Variable"] == "Subjektive Leistungseinschätzung") & (df["Category"] == "Indifferent (%)")]
    negativ_row = df[(df["Variable"] == "Subjektive Leistungseinschätzung") & (df["Category"] == "Negativ Einschätzung (%)")]

    # Extrahiere die Gesamtanzahl der Studierenden pro Studiengang
    total_students_row = df[(df["Variable"] == "Subjektive Leistungseinschätzung") & (df["Category"] == "Anzahl")]

    if positiv_row.empty or indifferent_row.empty or negativ_row.empty or total_students_row.empty:
        print(f"Warnung: Fehlende Daten für {jahr}. Überspringe...")
        continue  

    # Gemeinsame Studiengangs-Spalten bestimmen
    common_columns = positiv_row.columns.intersection(total_students_row.columns)[2:]

    # Werte als Float konvertieren und NaN durch 0 ersetzen
    positiv_values = positiv_row[common_columns].astype(float).fillna(0).squeeze()
    indifferent_values = indifferent_row[common_columns].astype(float).fillna(0).squeeze()
    negativ_values = negativ_row[common_columns].astype(float).fillna(0).squeeze()
    total_students_values = total_students_row[common_columns].astype(float).fillna(0).squeeze()

    # Absolute Anzahl für jede Kategorie berechnen
    abs_positiv = (positiv_values / 100) * total_students_values
    abs_indifferent = (indifferent_values / 100) * total_students_values
    abs_negativ = (negativ_values / 100) * total_students_values

    # Gesamtsummen für gewichtete Berechnung
    total_abs_positiv = abs_positiv.sum()
    total_abs_indifferent = abs_indifferent.sum()
    total_abs_negativ = abs_negativ.sum()
    total_students = total_students_values.sum()

    # Gewichtete Anteile berechnen
    gewichteter_positiv = (total_abs_positiv / total_students) * 100 if total_students > 0 else 0
    gewichteter_indifferent = (total_abs_indifferent / total_students) * 100 if total_students > 0 else 0
    gewichteter_negativ = (total_abs_negativ / total_students) * 100 if total_students > 0 else 0

    # Speichern der Ergebnisse
    leistung_df[jahr] = {
        "Gewichteter Positiv (%)": gewichteter_positiv,
        "Gewichteter Indifferent (%)": gewichteter_indifferent,
        "Gewichteter Negativ (%)": gewichteter_negativ
    }

# DataFrame erstellen
df_leistung = pd.DataFrame.from_dict(leistung_df, orient="index").reset_index()
df_leistung.rename(columns={"index": "Jahr"}, inplace=True)
df_leistung = df_leistung.sort_values(by="Jahr")

In [12]:
import plotly.io as pio

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

In [ ]:
import plotly.express as px

# DataFrame für Plot umstrukturieren
df_leistung_melted = df_leistung.melt(id_vars="Jahr", var_name="Einschätzung", value_name="Prozent")

# Farben für die drei Kategorien definieren
color_map = {
    "Gewichteter Positiv (%)": "#2ca02c",   # Grün
    "Gewichteter Indifferent (%)": "#ffcc00",  # Gelb
    "Gewichteter Negativ (%)": "#d62728"   # Rot
}

# Gestapeltes Balkendiagramm erstellen
fig = px.bar(
    df_leistung_melted,
    x="Jahr",
    y="Prozent",
    color="Einschätzung",
    barmode="stack",
    title="Subjektive Leistungseinschätzung pro Jahr",
    color_discrete_map=color_map,  # Eigene Farben setzen
    template="infoviz"
)

# Layout-Anpassungen
fig.update_layout(
    yaxis=dict(range=[0, 100], title="Prozentuale Verteilung"),
    xaxis=dict(title="Jahr"),
    legend_title="Einschätzung"
)

fig.show()

In [ ]:
import plotly.express as px

# DataFrame umstrukturieren (nur 2021 und 2022)
df_leistung_filtered = df_leistung[df_leistung["Jahr"].isin(["2019", "2020", "2021", "2022", "2023", "2024"])]

# "Positiv", "Indifferent" und "Negativ" auswählen
df_leistung_melted = df_leistung_filtered.melt(id_vars="Jahr", var_name="Einschätzung", value_name="Prozent")
df_leistung_melted = df_leistung_melted[df_leistung_melted["Einschätzung"].isin(
    ["Gewichteter Positiv (%)", "Gewichteter Indifferent (%)", "Gewichteter Negativ (%)"]
)]

# Farben definieren (Positiv wird grau)
color_map = {
    "Gewichteter Positiv (%)": "#A9A9A9",   # Grau
    "Gewichteter Indifferent (%)": "#ffcc00",  # Gelb
    "Gewichteter Negativ (%)": "#d62728"   # Rot
}

# Linien-Diagramm erstellen
fig = px.line(
    df_leistung_melted,
    x="Jahr",
    y="Prozent",
    color="Einschätzung",
    markers=False,
    title="Veränderung der subjektiven Leistungseinschätzung (2021–2022)",
    color_discrete_map=color_map,
    template="infoviz"
)

# **Gestrichelte vertikale Linien für 2021 und 2022**
for jahr in ["2021", "2022"]:
    fig.add_shape(
        type="line",
        x0=jahr, x1=jahr,  # X-Wert bleibt konstant für vertikale Linie
        y0=0, y1=100,  # Linie von unten nach oben durchziehen
        line=dict(color="grey", width=2, dash="dash"),  # Schwarz, gestrichelt                                                                                                                                                                                                                                                                                                                                                                                                  
    )

fig.add_annotation(
    x="2021",
    y=50,
    text=f"{df_leistung_filtered.query('Jahr == 2021')}"
)

# Layout-Anpassungen
fig.update_traces(line=dict(width=5))  # Linien dicker machen

fig.update_layout(
    xaxis=dict(title="Jahr", tickmode="array", tickvals=["2019", "2020", "2021", "2022", "2023", "2024"]),
    yaxis=dict(title="Prozentuale Verteilung", range=[0, 75]),
    legend_title="Einschätzung"
)

fig.show()

In [ ]:
import pandas as pd

# Daten für 2021 und 2021 filtern
df_2021 = df_leistung[df_leistung["Jahr"] == "2021"]
df_2021 = df_leistung[df_leistung["Jahr"] == "2021"]


In [ ]:
# Berechnung der Veränderung
veränderung_positiv = df_2021["Gewichteter Positiv (%)"].values[0] - df_2021["Gewichteter Positiv (%)"].values[0]
veränderung_indifferent = df_2021["Gewichteter Indifferent (%)"].values[0] - df_2021["Gewichteter Indifferent (%)"].values[0]
veränderung_negativ = df_2021["Gewichteter Negativ (%)"].values[0] - df_2021["Gewichteter Negativ (%)"].values[0]

# DataFrame zur Übersicht
df_veränderung = pd.DataFrame({
    "Kategorie": ["Positiv", "Indifferent", "Negativ"],
    "Veränderung (%)": [veränderung_positiv, veränderung_indifferent, veränderung_negativ]
})

In [23]:
df_veränderung

,Kategorie,Veränderung (%)
0,Positiv,13.216291
1,Indifferent,27.699344
2,Negativ,-40.998031


In [70]:
import pandas as pd
import plotly.express as px

# Sicherstellen, dass 2024 in den Daten vorhanden ist
jahr = "2024"

df_2024 = sheets[jahr]

# Studiengänge automatisch erkennen
studiengaenge = [col for col in df_2024.columns if "Bachelor" in col or "Master" in col]

# Extrahiere die subjektive Leistungseinschätzung (Positiv, Indifferent, Negativ)
positiv_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                        (df_2024["Category"] == "Positiv Einschätzung (%)")][studiengaenge]

indifferent_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                            (df_2024["Category"] == "Indifferent (%)")][studiengaenge]

negativ_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                        (df_2024["Category"] == "Negativ Einschätzung (%)")][studiengaenge]

# DataFrame zur Visualisierung vorbereiten
df_studiengang_2024 = pd.concat([positiv_2024, indifferent_2024, negativ_2024], keys=["Positiv", "Indifferent", "Negativ"])
df_studiengang_2024 = df_studiengang_2024.T.reset_index()
df_studiengang_2024.columns = ["Studiengang", "Positiv", "Indifferent", "Negativ"]

# **Studiengänge ohne Werte entfernen (Zeilen mit NaN oder nur 0 entfernen)**
df_studiengang_2024 = df_studiengang_2024.dropna().replace(0, pd.NA).dropna()

# **Nach positiver Einschätzung absteigend sortieren**
df_studiengang_2024 = df_studiengang_2024.sort_values(by="Positiv", ascending=False)

# DataFrame umstrukturieren für gestapeltes Balkendiagramm
df_melted = df_studiengang_2024.melt(id_vars="Studiengang", var_name="Einschätzung", value_name="Prozent")

# Farben definieren
color_map = {
    "Positiv": "#2ca02c",   # Grün
    "Indifferent": "#ffcc00",  # Gelb
    "Negativ": "#d62728"   # Rot
}

# Gestapeltes Balkendiagramm erstellen
fig = px.bar(
    df_melted,
    x="Studiengang",
    y="Prozent",
    color="Einschätzung",
    barmode="stack",
    title="Subjektive Leistungseinschätzung nach Studiengang (2024)",
    color_discrete_map=color_map,
    template="infoviz"
)

# Layout-Anpassungen
fig.update_layout(
    xaxis=dict(title="Studiengang", tickangle=-45),
    yaxis=dict(title="Prozentuale Verteilung", range=[0, 100]),
    legend_title="Einschätzung"
)

fig.show()

In [72]:
import pandas as pd
import plotly.express as px

# Sicherstellen, dass 2024 in den Daten vorhanden ist
jahr = "2021"

df_2024 = sheets[jahr]

# Studiengänge automatisch erkennen
studiengaenge = [col for col in df_2024.columns if "Bachelor" in col or "Master" in col]

# Extrahiere die subjektive Leistungseinschätzung (Positiv, Indifferent, Negativ)
positiv_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                        (df_2024["Category"] == "Positiv Einschätzung (%)")][studiengaenge]

indifferent_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                            (df_2024["Category"] == "Indifferent (%)")][studiengaenge]

negativ_2024 = df_2024[(df_2024["Variable"] == "Subjektive Leistungseinschätzung") & 
                        (df_2024["Category"] == "Negativ Einschätzung (%)")][studiengaenge]

# DataFrame zur Visualisierung vorbereiten
df_studiengang_2024 = pd.concat([positiv_2024, indifferent_2024, negativ_2024], keys=["Positiv", "Indifferent", "Negativ"])
df_studiengang_2024 = df_studiengang_2024.T.reset_index()
df_studiengang_2024.columns = ["Studiengang", "Positiv", "Indifferent", "Negativ"]

# **Studiengänge ohne Werte entfernen (Zeilen mit NaN oder nur 0 entfernen)**
df_studiengang_2024 = df_studiengang_2024.dropna().replace(0, pd.NA).dropna()

# **Nach positiver Einschätzung absteigend sortieren**
df_studiengang_2024 = df_studiengang_2024.sort_values(by="Positiv", ascending=False)

# DataFrame umstrukturieren für gestapeltes Balkendiagramm
df_melted = df_studiengang_2024.melt(id_vars="Studiengang", var_name="Einschätzung", value_name="Prozent")

# Farben definieren
color_map = {
    "Positiv": "#2ca02c",   # Grün
    "Indifferent": "#ffcc00",  # Gelb
    "Negativ": "#d62728"   # Rot
}

# Gestapeltes Balkendiagramm erstellen
fig = px.bar(
    df_melted,
    x="Studiengang",
    y="Prozent",
    color="Einschätzung",
    barmode="stack",
    title="Subjektive Leistungseinschätzung nach Studiengang (2021)",
    color_discrete_map=color_map,
    template="infoviz"
)

# Layout-Anpassungen
fig.update_layout(
    xaxis=dict(title="Studiengang", tickangle=-45),
    yaxis=dict(title="Prozentuale Verteilung", range=[0, 100]),
    legend_title="Einschätzung"
)

fig.show()